# a. Divide the dataset as train, development and test. 

In [74]:
import os
import pandas as pd
from sklearn.utils import shuffle

def load_all_txt(direction):
    allfiles = []
    contents = os.listdir(direction)
    for i in range(0,len(contents)):
        path = os.path.join(direction,contents[i])
        if os.path.isdir(path):
            allfiles.extend(load_all_txt(path))
        if os.path.isfile(path):
            allfiles.append(path)
    return allfiles

train_list = load_all_txt('/Users/jizhimeicrc/Desktop/data/train')
test_list = load_all_txt('/Users/jizhimeicrc/Desktop/data/test')

def readfile(list):
    new_sentences = []
    for e in list:
        f = open(e,'r')
        line = f.readline()
        new_sentences.append(line)
    return new_sentences

label = []
for i in range(25000):
    if i <= 12499:
        label.append(0)
    else:
        label.append(1)

sentences_train = readfile(train_list)
sentences_test = readfile(test_list)
train_dev = pd.DataFrame({'text':sentences_train,'label':label})
train_dev = shuffle(train_dev)

import re
import string

def clean_text(text):
    # Make text lowercase, remove text in square brackets,remove links,remove punctuation
    # remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


# Applying the cleaning function to both test and training datasets
train_dev['text'] = train_dev['text'].apply(lambda x: clean_text(x))

import nltk
from nltk.corpus import stopwords

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

### divide the data as train and dev, train:dev = 4:1
train = train_dev[:int(0.8*len(train_dev))]
# train = train_dev[:2000]
train = pd.DataFrame(train)
dev = train_dev[int(0.8*len(train_dev)):]
# dev = train_dev[2000:2400]
dev = pd.DataFrame(dev)
# tokenization
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x))
dev['text'] = dev['text'].apply(lambda x: tokenizer.tokenize(x))
train['text'] = train['text'].apply(lambda x: remove_stopwords(x))
dev['text'] = dev['text'].apply(lambda x: remove_stopwords(x))
print("train data:")
print(train.head())
print("dev data:")
print(dev.head())
print("\n")

test = pd.DataFrame({'text':sentences_test,'label':label})
test['text'] = test['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: tokenizer.tokenize(x))
test['text'] = test['text'].apply(lambda x: remove_stopwords(x))
print("test data:")
print(test.head())

train data:
                                                    text  label
19524  [year, ago, watching, tv, news, story, broadca...      1
19278  [movie, perfect, romantics, world, john, ritte...      1
1240   [personally, think, show, looks, pretty, cheap...      0
15820  [miss, company, vestron, sure, finger, pulse, ...      1
19813  [pretty, memorable, movie, animalskillingpeopl...      1
dev data:
                                                    text  label
19927  [know, thats, expect, film, sort, oflineage, d...      1
20581  [first, dont, go, revolver, expecting, another...      1
697    [really, wanted, like, movie, imposable, actin...      0
9628   [seriously, american, frech, actors, pretendin...      0
14311  [let, upfront, like, pulp, however, like, one,...      1


test data:
                                                text  label
0  [alan, rickman, emma, thompson, give, good, pe...      0
1  [seen, movie, care, movie, anyhow, would, thin...      0
2  [los, angeles,

# b. Build a vocabulary as list. 
[‘the’ ‘I’ ‘happy’ … ] 
You may omit rare words for example if the occurrence is less than five times
A reverse index as the key value might be handy
{“the”: 0, “I”:1, “happy”:2 , … }

In [75]:
def count_words(s):
    s_list = s.lower().split(' ')
    # calculate the occurrence of every word
    top_n_dict = {}
    for word in s_list:
        if word in top_n_dict:
            top_n_dict[word] += 1
        else:
            top_n_dict[word] = 1
    return top_n_dict

all_s = ""
for index, row in train.iterrows():
    s = " ".join(row['text'])
    all_s = all_s + s
train_voc_list = count_words(all_s)

for key, value in list(train_voc_list.items()):
    if value==5 or value==4 or value==3 or value==2 or value==1:
        del train_voc_list[key]
# print(train_voc_list)
print("vocabulary list:")
print(list(train_voc_list.items())[:50])

vocabulary list:
[('year', 1413), ('ago', 794), ('watching', 3390), ('tv', 2024), ('news', 247), ('story', 9040), ('broadcast', 85), ('zombie', 567), ('movie', 31692), ('filmed', 568), ('area', 258), ('since', 2231), ('paid', 273), ('particular', 583), ('attention', 676), ('called', 1020), ('fido', 60), ('finished', 204), ('production', 1339), ('began', 244), ('playing', 1267), ('festivals', 42), ('two', 5217), ('weeks', 157), ('local', 689), ('theater', 604), ('yesterday', 73), ('read', 1416), ('newspaper', 93), ('article', 40), ('stated', 110), ('attracting', 9), ('audiences', 410), ('limited', 240), ('release', 578), ('exception', 280), ('fact', 2761), ('shows', 1913), ('paramount', 51), ('including', 814), ('course', 1951), ('makes', 3339), ('sense', 1797), ('many', 5155), ('locals', 52), ('want', 2909), ('see', 8791), ('city', 877), ('screen', 1686), ('spot', 269)]


# c. Calculate the following probability
Probability of the occurrence
P[“the”] = num of documents containing ‘the’ / num of all documents
Conditional probability based on the sentiment
P[“the” | Positive]  = # of positive documents containing “the” / num of all positive review documents

In [76]:
total_length = len(train)
train_list = pd.DataFrame({'text':train['text'].apply(lambda x: count_words(" ".join(x))), 'label':train['label']})

def cal_total():
    pos = 0
    neg = 0
    for index, row in train.iterrows():
        if row['label'] == 0:
            neg = neg+1
        if row['label'] == 1:
            pos = pos+1
    return pos, neg

total_pos, total_neg = cal_total()

def cal_prob():
    prob = {}
    con_p_pos = {}
    con_p_neg = {}
    for key in train_voc_list:
        count = 0
        count_pos = 0
        count_neg = 0
        for index, row in train_list.iterrows():
            if key in row['text']:
                count += 1
                if row['label'] == 0:
                    count_neg += 1
                if row['label'] == 1:
                    count_pos += 1
        prob[key] = count/total_length;
        con_p_pos[key] = count_pos/total_pos
        con_p_neg[key] = count_neg/total_neg
    return prob, con_p_pos, con_p_neg

probability, conditional_p_pos, conditional_p_neg = cal_prob()

print("prior probability:")
print(list(probability.items())[:20])

print("\nconditional probability:")
print("positive:\n", list(conditional_p_pos.items())[:20])
print("\n")
print("negative:\n", list(conditional_p_neg.items())[:20])

prior probability:
[('year', 0.0621), ('ago', 0.038), ('watching', 0.14945), ('tv', 0.0793), ('news', 0.01105), ('story', 0.29605), ('broadcast', 0.00405), ('zombie', 0.0126), ('movie', 0.59835), ('filmed', 0.0267), ('area', 0.01115), ('since', 0.0979), ('paid', 0.01355), ('particular', 0.0277), ('attention', 0.03215), ('called', 0.04575), ('fido', 0.0011), ('finished', 0.0112), ('production', 0.05615), ('began', 0.01175)]

conditional probability:
positive:
 [('year', 0.07034675169390195), ('ago', 0.04453965723395775), ('watching', 0.12774013551215624), ('tv', 0.07811877241929055), ('news', 0.010661618174571542), ('story', 0.32781984854523716), ('broadcast', 0.0050817058589079315), ('zombie', 0.007174172977281785), ('movie', 0.5516141889198884), ('filmed', 0.027899561578318056), ('area', 0.011658031088082901), ('since', 0.10253088880031885), ('paid', 0.006974890394579514), ('particular', 0.033778397768035075), ('attention', 0.03626943005181347), ('called', 0.041550418493423676), ('fid

# d. Calculate accuracy using dev dataset 
Conduct five fold cross validation

## d.1 calculate accuracy using dev data before

In [77]:
print(dev.head())

def classify(s):
    pos_p = total_pos/len(train)
    neg_p = total_neg/len(train)
    for word in s:
        if word in conditional_p_pos:
            pos_p = pos_p*conditional_p_pos[word]
        if word in conditional_p_neg:
            neg_p = neg_p*conditional_p_neg[word]
    if(pos_p>=neg_p):
        return 1
    if(pos_p<neg_p):
        return 0

right_num = 0
for index, row in dev.iterrows():
    if row['label']==classify(row['text']):
        right_num += 1
        
accuracy = right_num/len(dev)
print("\n*********predict accuracy*********")
print(accuracy)    

                                                    text  label
19927  [know, thats, expect, film, sort, oflineage, d...      1
20581  [first, dont, go, revolver, expecting, another...      1
697    [really, wanted, like, movie, imposable, actin...      0
9628   [seriously, american, frech, actors, pretendin...      0
14311  [let, upfront, like, pulp, however, like, one,...      1

*********predict accuracy*********
0.7982


## d.2 cross validation

In [58]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(5, True, 10)
train_dev_new = train_dev[:1000]
train_dev_new = train_dev_new.values
result = 0

for train_idx, dev_idx in kf.split(train_dev_new):
    train_new = train_dev_new[train_idx]
    dev_new = train_dev_new[dev_idx]
    train_new = pd.DataFrame(train_new)
    dev_new = pd.DataFrame(dev_new)
    train_new.rename(columns={0:'text'},inplace=True)
    train_new.rename(columns={1:'label'},inplace=True)
    dev_new.rename(columns={0:'text'},inplace=True)
    dev_new.rename(columns={1:'label'},inplace=True)

    for index, row in train_new.iterrows():
        s1 = " ".join(row['text'])
        row['text'] = clean_text(s1)
        row['text'] = tokenizer.tokenize(row['text'])
        row['text'] = remove_stopwords(row['text'])
        
    for index, row in dev_new.iterrows():
        s1 = " ".join(row['text'])
        row['text'] = clean_text(s1)
        row['text'] = tokenizer.tokenize(row['text'])
        row['text'] = remove_stopwords(row['text'])
    
    all_s = ""
    for index, row in train_new.iterrows():
        s = " ".join(row['text'])
        all_s = all_s + s
    new_list = count_words(all_s)

    for key, value in list(new_list.items()):
        if value==5 or value==4 or value==3 or value==2 or value==1:
            del new_list[key]
            
    total_length = len(train_new)
    train_new_list = pd.DataFrame({'text':train_new['text'].apply(lambda x: count_words(" ".join(x))), 'label':train_new['label']})
    total_pos_new, total_neg_new = cal_total()
    probability_new, conditional_p_pos_new, conditional_p_neg_new = cal_prob()
    
    right_num_new = 0
    for index, row in dev_new.iterrows():
        if row['label']==classify(row['text']):
            right_num_new += 1
        
    accuracy_new = right_num_new/len(dev_new)
    print("accuracy: ", accuracy_new, "\n")
    result += accuracy_new

print("\n*********predict accuracy*********")
print(result/5) 

accuracy:  0.95 

accuracy:  0.93 

accuracy:  0.93 

accuracy:  0.955 

accuracy:  0.925 


*********predict accuracy*********
0.9380000000000001


# e. Do following experiments
Compare the effect of Smoothing
Derive Top 10 words that predicts positive and negative class
P[Positive| word] 

In [110]:
pos_word = {}
neg_word = {}

for key1 in conditional_p_pos:
    if key1 in probability:
        pos_word[key1] = (conditional_p_pos[key1]+1)/(probability[key1]+len(conditional_p_pos))
a1 = sorted(pos_word.items(),key = lambda x:x[1], reverse = True)
    
for key2 in conditional_p_neg:
    if key2 in probability:
        neg_word[key2] = (conditional_p_neg[key2]+1)/(probability[key2]+len(conditional_p_neg))
a2 = sorted(neg_word.items(),key = lambda x:x[1], reverse = True)

print("Top 10 of P[Positive| word]:")
for element in a1[:10]:
    print(element[0])
print("\nTop 10 of P[Negative| word]:")
for element in a2[:10]:
    print(element[0])

Top 10 of P[Positive| word]:
one
movie
film
like
good
great
time
story
see
well

Top 10 of P[Negative| word]:
movie
one
film
like
even
good
would
bad
time
really


# f. Using the test dataset
Use the optimal hyperparameters you found in the step e, and use it to calculate the final accuracy.  

In [101]:
pred_right = 0
for index, row in test.iterrows():
    if row['label']==classify(row['text']):
        pred_right += 1
accu = pred_right/len(test)
print("*********predict accuracy*********")
print(accu)

*********predict accuracy*********
0.77736
